In [8]:
import requests
from bs4 import BeautifulSoup
import re, json, os, random
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce 
import pandas as pd
from io import StringIO

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")
nn_id = "nn123"
nn_wf_ver_id ="1"

print("import common done")
import re 

data = "aaabbbcccddsgjs adjkfeklsjdfk jvsklfjsdklf jsdfadffsdf"
tel_no_list = "010-9999-9999, 019-2222-4444, 082-1111-3333, 112, 02-111-1111, 3333-3333"
virtual_html = "<html><body><title>abcd</title><p>가나다라</p><p>마바사아</p></body></html>"

#알파뱃(a~z) 중에서 단어를 찾는다 
reg = re.compile('[a-z]*')
print("1.알파뱃 찾기")
print("IN : {0}".format(reg.findall(data)))
print("OUT : {0}".format(reg.findall(data)))
print('\n')

#알파뱃(a~z) 중에서 두단어를 찾는다 
reg = re.compile('[a-z]{0,5}')
print("2.알파뱃 두단어 찾기")
print("IN : {0}".format(reg.findall(data)))
print("OUT : {0}".format(reg.findall(data)))
print('\n')

# 특정 패턴에 해당하는 전화번호를 다 찾아보자 
reg = re.compile('\d{2,3}-\d{3,4}-\d{4,4}')
print("3.전화번호 패턴 추출")
print("IN : {0}".format(tel_no_list))
print("OUT : {0}".format(reg.findall(tel_no_list)))
print('\n')

# 특정 패턴에 해당하는 전화번호를 다 찾아보자 
# 3333-3333 도 찾아보자 
reg = re.compile('(\d{2,3}-\d{3,4}-\d{4,4}|\d{3,4}-\d{4,4})')
out = reg.findall(tel_no_list)
print("4.전화번호 패턴 추출 (or 문 사용)")
print("IN : {0}".format(tel_no_list))
print("OUT : {0}".format(reg.findall(tel_no_list)))
print('\n')

# 특정 문자를 제외하고 
reg = re.compile('[^a]{1,1}[\w]+$')
print("5.특정 패턴 제외하고 찾기")
print("IN : {0}".format(data))
print("OUT : {0}".format(reg.findall(data)))
print('\n')

# 한글 전체 찾기
reg = re.compile('[가-힣]{1,}')
print("5.한글만 다 찾아 보기")
print("IN : {0}".format(virtual_html))
print("OUT : {0}".format(reg.findall(virtual_html)))
print('\n')

# title 태그 안에 있는 것만 다 가지고 와보자
# (xxx) 은 추출이다 
reg = re.compile('<title[^>]*>([^<]+)</title>')
print("6.Title Tag만 다 찾아보기")
print("IN : {0}".format(virtual_html))
print("OUT : {0}".format(reg.findall(virtual_html)))
print('\n')

# p 태그 안에 있는 것만 다 가지고 와보자
reg = re.compile('<p>([\w]+)</p>')
print("7.P Tag만 다 찾아보기")
print("IN : {0}".format(virtual_html))
print("OUT : {0}".format(reg.findall(virtual_html)))
print('\n')

import common done
1.알파뱃 찾기
IN : ['aaabbbcccddsgjs', '', 'adjkfeklsjdfk', '', 'jvsklfjsdklf', '', 'jsdfadffsdf', '']
OUT : ['aaabbbcccddsgjs', '', 'adjkfeklsjdfk', '', 'jvsklfjsdklf', '', 'jsdfadffsdf', '']


2.알파뱃 두단어 찾기
IN : ['aaabb', 'bcccd', 'dsgjs', '', 'adjkf', 'eklsj', 'dfk', '', 'jvskl', 'fjsdk', 'lf', '', 'jsdfa', 'dffsd', 'f', '']
OUT : ['aaabb', 'bcccd', 'dsgjs', '', 'adjkf', 'eklsj', 'dfk', '', 'jvskl', 'fjsdk', 'lf', '', 'jsdfa', 'dffsd', 'f', '']


3.전화번호 패턴 추출
IN : 010-9999-9999, 019-2222-4444, 082-1111-3333, 112, 02-111-1111, 3333-3333
OUT : ['010-9999-9999', '019-2222-4444', '082-1111-3333', '02-111-1111']


4.전화번호 패턴 추출 (or 문 사용)
IN : 010-9999-9999, 019-2222-4444, 082-1111-3333, 112, 02-111-1111, 3333-3333
OUT : ['010-9999-9999', '019-2222-4444', '082-1111-3333', '02-111-1111', '3333-3333']


5.특정 패턴 제외하고 찾기
IN : aaabbbcccddsgjs adjkfeklsjdfk jvsklfjsdklf jsdfadffsdf
OUT : [' jsdfadffsdf']


5.한글만 다 찾아 보기
IN : <html><body><title>abcd</title><p>가나다라</p><p>마바사아</p></bod

In [9]:
def crawler(iter) : 
    url = "http://naver.com"
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text , 'lxml')
    for raw in soup.find_all('title') : 
        print(raw)
crawler(1)

<title>NAVER</title>


In [10]:
def task(page, max_pages, url_path, file_w, reg = None):
    """
    지정된 수만큼 제귀 형태로 모든 링크를 따라가서 전부 수집한다. 
    """
    if page == max_pages :
        get_single_article(url_path, file_w, reg_exp=str(reg))
        table_to_csv(url_path)
    else : 
        get_single_article(url_path, file_w, reg_exp=str(reg))
        table_to_csv(url_path)
        source_code = requests.get(url_path)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'lxml')
        page += 1
        for link in soup.find_all('a'):
            href = link.get('href')
            if (href != None and re.search("https://ko", href)) : 
                task(page, max_pages, href, file_w, reg=str(reg))

def get_single_article(item_url, file_w, reg_exp = None):
    """
    p 태그를 가지고와서 파싱하거나 
    지정된 reg_exp 를 사용하여 파싱한다 
    """
    print("href : {0}".format(item_url))
    source_code = requests.get(item_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    
    if(reg_exp) : 
        #정규 표현식이 있는 경우 해당 정규 표현식에 맞는 데이터를 추출 
        reg = re.compile(reg_exp)
        for contents in reg.findall(plain_text):
            file_w.write(contents)
    else : 
        #별도의 Regex가 없는 경우 p tag 에 있는 모든 데이터 추출 
        for contents in soup.find_all('p'):
            file_w.write(contents.text)

def spider(max_pages, url_path, path = "/home/dev/wiki/", file_name='test.txt', reg_exp = None) :
    """
    본 Function 을 실행하면 WikiPedia 첫 페이지에서 실행해서 
    지정된 횟수만큼 페이지를 따라 들어가서 정해진 패턴을 수집한다. 
    max_pages : 몇번 Page를 따라 들어갈 것인가를 정의하는 변수 
    """
    if not os.path.exists(path):
        os.makedirs(path)
    with open(''.join([path, file_name]), "w") as file_w :   
        print("# Job Start!!")
        task(1, max_pages, url_path, file_w, reg = reg_exp)
        print("# Job Done!!")

def save_as_csv(data) :
    """
    랜덤하게 csv 이름을 생성하여 주어진 데이터를 저장 
    """
    rand_name = random.randrange(1,10000)
    save_data = StringIO(data)
    df = pd.read_csv(save_data, sep=",")
    df.to_csv("/home/dev/csv/" + str(rand_name) + ".csv", sep=',', encoding='utf-8')
    print("file saved as : {0}".format(str(rand_name)))
    
def table_to_csv(url) : 
    """
    전형적인 형태의 Table 을 Parsing 하여 csv 로 저장한다 
    """
    try : 
        return_line = []
        return_td = ""
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'lxml')
        for table in soup.find_all('table') :
            for tr in table.find_all('tr') :
                for td in table.find_all('td') :
                    return_td = return_td + td.text + ','
                return_td = return_td.rstrip(',') + '\\n'
            # save each table 
            save_as_csv(return_td)
            return_td = ""
    except Exception as e : 
        return True
        
# 주어진 횟수만큼 해당 사이트를 시작으로 크롤링 시작 
# first parm : Inception 횟수 
# second parm : initial site 
# reg_exp : 정규 표현식 사용 가능 

# (1) 정규 표현식 사용 CASE (한글 전체 추출)
spider(2, 'https://ko.wikipedia.org/wiki/', reg_exp ='[가-힣\s]{1,}')
#spider(2, 'https://ko.wikipedia.org/wiki/', reg_exp ='<title[^>]*>([^<]+)</title>')

# (2) 정규 표현식 사용하지 않고 P 태크 추출
#spider(2, 'https://ko.wikipedia.org/wiki/')

# Job Start!!
href : https://ko.wikipedia.org/wiki/
href : https://ko.wiktionary.org/wiki/
href : https://ko.wiktionary.org/wiki/
href : https://ko.wikinews.org/wiki/
href : https://ko.wikinews.org/wiki/
href : https://ko.wikisource.org/wiki/
href : https://ko.wikisource.org/wiki/
href : https://ko.wikiversity.org/wiki/
href : https://ko.wikiversity.org/wiki/
href : https://ko.wikivoyage.org/wiki/
href : https://ko.wikivoyage.org/wiki/%EB%8C%80%EB%AC%B8
href : https://ko.wikiquote.org/wiki/
href : https://ko.wikiquote.org/wiki/
href : https://ko.wikibooks.org/wiki/
href : https://ko.wikibooks.org/wiki/
href : https://ko.wikipedia.org/w/index.php?title=위키백과:대문&oldid=15252069
href : https://ko.wikibooks.org/wiki/%EC%9C%84%ED%82%A4%EC%B1%85:%EB%8C%80%EB%AC%B8
href : https://ko.wikinews.org/wiki/%EC%9C%84%ED%82%A4%EB%89%B4%EC%8A%A4:%EB%8C%80%EB%AC%B8
href : https://ko.wikiquote.org/wiki/%EC%9C%84%ED%82%A4%EC%9D%B8%EC%9A%A9%EC%A7%91:%EB%93%A4%EB%A8%B8%EB%A6%AC
href : https://ko.wikisource.o